In [12]:
"""
PROCESSAMENTO: INTENSIDADE 4G - ANATEL (COM DE-PARA IBGE-TSE)
Autor: Fabio
Data: 2024-12-05

Versão CORRIGIDA usando tabela de-para oficial IBGE ↔ TSE
"""

import pandas as pd
import numpy as np
from pathlib import Path

# ============================================================================
# CONFIGURAÇÕES
# ============================================================================

# Arquivos - AJUSTE OS CAMINHOS CONFORME NECESSÁRIO
ARQUIVO_ANATEL = r"C:\Users\fabio\OneDrive\EconomiaPolítica\4g-Eleicoes\4G-Eleições\quantitativo_ERB.xlsx"
ARQUIVO_PAINEL = r"C:\Users\fabio\OneDrive\EconomiaPolítica\4g-Eleicoes\4G-Eleições\painel_event_study_com_3g_final.csv"
ARQUIVO_DEPARA = r"C:\Users\fabio\OneDrive\EconomiaPolítica\4g-Eleicoes\4G-Eleições\depara-ibge-tse.csv"

# Output
OUTPUT_FILE = r"C:\Users\fabio\OneDrive\EconomiaPolítica\4g-Eleicoes\4G-Eleições\painel_com_intensidade_4g.csv"

In [13]:
# ============================================================================
# 1. CARREGAR TABELA DE-PARA IBGE-TSE
# ============================================================================

print("📋 Carregando tabela de-para IBGE-TSE...")

depara = pd.read_csv(
    ARQUIVO_DEPARA,
    sep=';',
    encoding='latin1',
    dtype={
        'CD_MUNICIPIO_TSE': str,
        'CD_MUNICIPIO_IBGE': str
    }
)

print(f"✅ Tabela de-para carregada: {len(depara):,} correspondências")
print(f"   Colunas: {list(depara.columns)}")
print()

# Limpar e padronizar
depara['codigo_ibge'] = depara['CD_MUNICIPIO_IBGE'].str.strip()
depara['codigo_tse'] = depara['CD_MUNICIPIO_TSE'].str.strip().str.zfill(5)

print("📊 Exemplos da tabela de-para:")
print(depara[['codigo_ibge', 'codigo_tse', 'NM_MUNICIPIO_IBGE', 'SG_UF']].head(10))
print()

📋 Carregando tabela de-para IBGE-TSE...
✅ Tabela de-para carregada: 5,571 correspondências
   Colunas: ['DT_GERACAO', 'HH_GERACAO', 'CD_UF_TSE', 'CD_UF_IBGE', 'SG_UF', 'NM_UF', 'CD_MUNICIPIO_TSE', 'NM_MUNICIPIO_TSE', 'CD_MUNICIPIO_IBGE', 'NM_MUNICIPIO_IBGE']

📊 Exemplos da tabela de-para:
  codigo_ibge codigo_tse     NM_MUNICIPIO_IBGE SG_UF
0     1200138      01007                Bujari    AC
1     1200179      01015              Capixaba    AC
2     1200807      01023            Porto Acre    AC
3     1200435      01031   Santa Rosa Do Purus    AC
4     1200351      01040  Marechal Thaumaturgo    AC
5     1200104      01058             Brasiléia    AC
6     1200393      01066          Porto Walter    AC
7     1200203      01074       Cruzeiro Do Sul    AC
8     1200427      01082       Rodrigues Alves    AC
9     1200336      01090           Mâncio Lima    AC



In [14]:
# ============================================================================
# 2. CARREGAR DADOS ANATEL
# ============================================================================

print("📊 Carregando dados ANATEL (XLSX)...")

df_anatel = pd.read_excel(
    ARQUIVO_ANATEL,
    sheet_name=0,
    engine='openpyxl'
)

print(f"✅ Arquivo XLSX carregado: {len(df_anatel):,} municípios")
print()

# Limpar nomes de colunas
df_anatel.columns = df_anatel.columns.str.strip().str.replace('\n', ' ').str.replace('\r', '')

# Identificar coluna de código IBGE
col_codigo = None
for col in df_anatel.columns:
    if 'CÓDIGO' in col.upper() and 'IBGE' in col.upper():
        col_codigo = col
        break

if col_codigo is None:
    raise Exception("❌ Coluna de código IBGE não encontrada!")

# Renomear e converter
df_anatel['codigo_ibge'] = df_anatel[col_codigo].astype(str).str.strip()

print(f"🔍 Código IBGE identificado na coluna: '{col_codigo}'")
print(f"   Exemplos: {df_anatel['codigo_ibge'].head(5).tolist()}")
print()

# Padronizar nomes de outras colunas importantes
column_mapping = {
    'NOME': 'municipio',
    'ESTADO SIGLA': 'uf',
    'POPULAÇÃO': 'populacao',
    'ÁREA DE ALTA DENSIDADE DE EDIFICAÇÕES': 'area_alta_densidade',
    'ÁREA DE BAIXA DENSIDADE DE EDIFICAÇÕES': 'area_baixa_densidade',
    'Quantidade de ERBs de cobertura - 700 MHz': 'erbs_700mhz',
    'Quantidade de ERBs de cobertura - 2,5 GHz': 'erbs_2500mhz',
}

for old, new in column_mapping.items():
    if old in df_anatel.columns:
        df_anatel.rename(columns={old: new}, inplace=True)

print("✅ Colunas padronizadas")
print()

📊 Carregando dados ANATEL (XLSX)...
✅ Arquivo XLSX carregado: 5,570 municípios

🔍 Código IBGE identificado na coluna: 'CÓDIGO IBGE'
   Exemplos: ['1100015', '1100023', '1100031', '1100049', '1100056']

✅ Colunas padronizadas



In [16]:
# ============================================================================
# 3. CONVERTER TIPOS E CRIAR VARIÁVEIS DE INTENSIDADE
# ============================================================================

print("🔧 Convertendo tipos e criando variáveis de intensidade...")

# Converter colunas numéricas
numeric_cols = ['populacao', 'area_alta_densidade', 'area_baixa_densidade', 
                'erbs_700mhz', 'erbs_2500mhz']

for col in numeric_cols:
    if col in df_anatel.columns:
        df_anatel[col] = pd.to_numeric(df_anatel[col], errors='coerce').fillna(0)

# 1. Total de ERBs 4G
df_anatel['total_erbs_4g'] = df_anatel['erbs_700mhz'] + df_anatel['erbs_2500mhz']

# 2. Tem 4G (dummy)
df_anatel['tem_4g'] = (df_anatel['total_erbs_4g'] > 0).astype(int)

# 3. ERBs por 1000 habitantes
df_anatel['erbs_per_1000hab'] = np.where(
    df_anatel['populacao'] > 0,
    df_anatel['total_erbs_4g'] / df_anatel['populacao'] * 1000,
    0
)

# 4. ERBs por km²
df_anatel['area_total'] = df_anatel['area_alta_densidade'] + df_anatel['area_baixa_densidade']
df_anatel['erbs_per_km2'] = np.where(
    df_anatel['area_total'] > 0,
    df_anatel['total_erbs_4g'] / df_anatel['area_total'],
    0
)

# 5. Intensidade normalizada (0-1)
max_intensidade = df_anatel['erbs_per_1000hab'].max()
df_anatel['intensidade_4g_norm'] = np.where(
    max_intensidade > 0,
    df_anatel['erbs_per_1000hab'] / max_intensidade,
    0
)

# 6. Categorias de intensidade
df_anatel['categoria_4g'] = pd.cut(
    df_anatel['erbs_per_1000hab'],
    bins=[0, 0.01, 0.1, 0.5, np.inf],
    labels=['Sem 4G', 'Baixa', 'Média', 'Alta'],
    include_lowest=True
)

# 7. Dummy: alta intensidade (acima da mediana)
mediana = df_anatel.loc[df_anatel['total_erbs_4g'] > 0, 'erbs_per_1000hab'].median()
df_anatel['alta_intensidade_4g'] = (df_anatel['erbs_per_1000hab'] > mediana).astype(int)

print("✅ Variáveis de intensidade criadas:")
print(f"   - total_erbs_4g: {df_anatel['total_erbs_4g'].sum():,.0f} ERBs")
print(f"   - Municípios com 4G: {df_anatel['tem_4g'].sum():,}")
print(f"   - ERBs/1000hab (média): {df_anatel['erbs_per_1000hab'].mean():.3f}")
print(f"   - Mediana intensidade: {mediana:.3f}")
print()

# Estatísticas por categoria
print("📊 Distribuição por categoria:")
print(df_anatel['categoria_4g'].value_counts().sort_index())
print()

🔧 Convertendo tipos e criando variáveis de intensidade...
✅ Variáveis de intensidade criadas:
   - total_erbs_4g: 34,725 ERBs
   - Municípios com 4G: 5,570
   - ERBs/1000hab (média): 0.378
   - Mediana intensidade: 0.295

📊 Distribuição por categoria:
categoria_4g
Sem 4G       2
Baixa      289
Média     3975
Alta      1304
Name: count, dtype: int64



In [17]:
# ============================================================================
# 4. MERGE ANATEL + DE-PARA (ADICIONAR CÓDIGO TSE)
# ============================================================================

print("="*80)
print("MERGE: ANATEL + DE-PARA (IBGE → TSE)")
print("="*80 + "\n")

print("🔗 Fazendo merge com tabela de-para...")

# Merge ANATEL com DE-PARA usando código IBGE
df_anatel_tse = df_anatel.merge(
    depara[['codigo_ibge', 'codigo_tse']],
    on='codigo_ibge',
    how='left'
)

# Verificar cobertura
n_matched = df_anatel_tse['codigo_tse'].notna().sum()
pct_matched = n_matched / len(df_anatel_tse) * 100

print(f"✅ Merge ANATEL + DE-PARA concluído")
print(f"   Municípios com código TSE: {n_matched:,} ({pct_matched:.1f}%)")
print(f"   Municípios sem código TSE: {(len(df_anatel_tse) - n_matched):,}")
print()

if pct_matched < 95:
    print(f"⚠️  Atenção: {100-pct_matched:.1f}% dos municípios não têm código TSE!")
    print("   Verificar se códigos IBGE estão corretos")
    print()

# Mostrar exemplos de municípios sem match
sem_match = df_anatel_tse[df_anatel_tse['codigo_tse'].isna()]
if len(sem_match) > 0:
    print("🔍 Exemplos de municípios sem correspondência TSE:")
    print(sem_match[['codigo_ibge', 'municipio', 'uf']].head(10))
    print()


MERGE: ANATEL + DE-PARA (IBGE → TSE)

🔗 Fazendo merge com tabela de-para...
✅ Merge ANATEL + DE-PARA concluído
   Municípios com código TSE: 5,570 (100.0%)
   Municípios sem código TSE: 0



In [19]:
# ============================================================================
# 5. MERGE COM PAINEL PRINCIPAL
# ============================================================================

print("="*80)
print("MERGE: PAINEL PRINCIPAL + INTENSIDADE 4G")
print("="*80 + "\n")

print("📊 Carregando painel principal...")

df_painel = pd.read_csv(
    ARQUIVO_PAINEL,
    dtype={'codigo_tse': str}
)

# Garantir que codigo_tse seja string com padding
df_painel['codigo_tse'] = df_painel['codigo_tse'].astype(str).str.strip().str.zfill(5)

print(f"   Linhas: {len(df_painel):,}")
print(f"   Municípios únicos: {df_painel['codigo_tse'].nunique():,}")
print(f"   Exemplos códigos TSE: {df_painel['codigo_tse'].drop_duplicates().head(5).tolist()}")
print()

print("🔗 Fazendo merge com painel...")

# Selecionar colunas relevantes para merge
colunas_merge = [
    'codigo_tse',
    'total_erbs_4g',
    'erbs_700mhz',
    'erbs_2500mhz',
    'erbs_per_1000hab',
    'erbs_per_km2',
    'intensidade_4g_norm',
    'categoria_4g',
    'alta_intensidade_4g',
    'area_alta_densidade',
    'area_baixa_densidade'
]

df_anatel_merge = df_anatel_tse[colunas_merge].copy()

# Merge
df_painel_final = df_painel.merge(
    df_anatel_merge,
    on='codigo_tse',
    how='left'
)

print(f"✅ Merge concluído")
print(f"   Linhas finais: {len(df_painel_final):,}")
print()

# Verificar cobertura
n_com_dados = df_painel_final['total_erbs_4g'].notna().sum()
pct_com_dados = n_com_dados / len(df_painel_final) * 100

print(f"📊 COBERTURA DO MERGE:")
print(f"   Observações com dados 4G: {n_com_dados:,} ({pct_com_dados:.1f}%)")
print(f"   Observações sem dados 4G: {(len(df_painel_final) - n_com_dados):,}")
print()


MERGE: PAINEL PRINCIPAL + INTENSIDADE 4G

📊 Carregando painel principal...
   Linhas: 16,489
   Municípios únicos: 4,123
   Exemplos códigos TSE: ['10022', '10049', '10057', '10073', '10081']

🔗 Fazendo merge com painel...
✅ Merge concluído
   Linhas finais: 16,489

📊 COBERTURA DO MERGE:
   Observações com dados 4G: 16,489 (100.0%)
   Observações sem dados 4G: 0



In [20]:
# ============================================================================
# 6. PREENCHER VALORES FALTANTES
# ============================================================================

print("🔧 Preenchendo valores faltantes...")

# Municípios sem dados ANATEL = sem 4G
fill_cols = ['total_erbs_4g', 'erbs_700mhz', 'erbs_2500mhz', 
             'erbs_per_1000hab', 'erbs_per_km2', 'intensidade_4g_norm',
             'alta_intensidade_4g']

for col in fill_cols:
    if col in df_painel_final.columns:
        df_painel_final[col] = df_painel_final[col].fillna(0)

if 'categoria_4g' in df_painel_final.columns:
    df_painel_final['categoria_4g'] = df_painel_final['categoria_4g'].fillna('Sem 4G')

print("✅ Valores preenchidos")
print()

🔧 Preenchendo valores faltantes...
✅ Valores preenchidos



In [21]:
# ============================================================================
# 7. ESTATÍSTICAS FINAIS
# ============================================================================

print("="*80)
print("ESTATÍSTICAS FINAIS")
print("="*80 + "\n")

# Por ano (se houver)
if 'ano' in df_painel_final.columns:
    print("📅 INTENSIDADE MÉDIA POR ANO:")
    stats_ano = df_painel_final.groupby('ano').agg({
        'erbs_per_1000hab': 'mean',
        'total_erbs_4g': ['sum', lambda x: (x > 0).sum()]
    }).round(3)
    stats_ano.columns = ['Média ERBs/1000hab', 'Total ERBs', 'N municípios com 4G']
    print(stats_ano)
    print()

# Distribuição geral
print("📊 DISTRIBUIÇÃO DE INTENSIDADE:")
print(f"   Mín: {df_painel_final['erbs_per_1000hab'].min():.3f}")
print(f"   Q25: {df_painel_final['erbs_per_1000hab'].quantile(0.25):.3f}")
print(f"   Mediana: {df_painel_final['erbs_per_1000hab'].median():.3f}")
print(f"   Q75: {df_painel_final['erbs_per_1000hab'].quantile(0.75):.3f}")
print(f"   Máx: {df_painel_final['erbs_per_1000hab'].max():.3f}")
print()

print("📊 POR CATEGORIA:")
print(df_painel_final['categoria_4g'].value_counts().sort_index())
print()

# Top 10 municípios com maior intensidade
print("🏆 TOP 10 MUNICÍPIOS - MAIOR INTENSIDADE (última observação):")
if 'ano' in df_painel_final.columns:
    ultimo_ano = df_painel_final['ano'].max()
    top10 = df_painel_final[df_painel_final['ano'] == ultimo_ano].nlargest(10, 'erbs_per_1000hab')
    if 'municipio' in top10.columns:
        print(top10[['municipio', 'uf', 'total_erbs_4g', 'erbs_per_1000hab']].to_string(index=False))
    else:
        print(top10[['codigo_tse', 'total_erbs_4g', 'erbs_per_1000hab']].to_string(index=False))
else:
    top10 = df_painel_final.nlargest(10, 'erbs_per_1000hab')
    print(top10[['codigo_tse', 'total_erbs_4g', 'erbs_per_1000hab']].to_string(index=False))
print()

ESTATÍSTICAS FINAIS

📅 INTENSIDADE MÉDIA POR ANO:
      Média ERBs/1000hab  Total ERBs  N municípios com 4G
ano                                                      
2010                0.38       27836                 4120
2014                0.38       27851                 4123
2018                0.38       27851                 4123
2022                0.38       27851                 4123

📊 DISTRIBUIÇÃO DE INTENSIDADE:
   Mín: 0.001
   Q25: 0.181
   Mediana: 0.290
   Q75: 0.491
   Máx: 3.257

📊 POR CATEGORIA:
categoria_4g
Sem 4G        4
Baixa       980
Média     11511
Alta       3994
Name: count, dtype: int64

🏆 TOP 10 MUNICÍPIOS - MAIOR INTENSIDADE (última observação):
codigo_tse  total_erbs_4g  erbs_per_1000hab
     92231              3          3.257329
     53317              2          2.341920
     85308             25          2.196065
     62430              2          2.155172
     92835              3          2.114165
     89508              7          2.088305
     

In [22]:
# ============================================================================
# 8. SALVAR RESULTADO
# ============================================================================

print("="*80)
print("SALVANDO RESULTADO")
print("="*80 + "\n")

df_painel_final.to_csv(OUTPUT_FILE, index=False)

print(f"✅ Arquivo salvo: {OUTPUT_FILE}")
print(f"📊 Linhas: {len(df_painel_final):,}")
print(f"📋 Colunas: {len(df_painel_final.columns)}")
print()

SALVANDO RESULTADO

✅ Arquivo salvo: C:\Users\fabio\OneDrive\EconomiaPolítica\4g-Eleicoes\4G-Eleições\painel_com_intensidade_4g.csv
📊 Linhas: 16,489
📋 Colunas: 43



In [23]:
# ============================================================================
# 9. RESUMO FINAL
# ============================================================================

print("="*80)
print("RESUMO FINAL")
print("="*80 + "\n")

print("✅ PROCESSAMENTO CONCLUÍDO COM SUCESSO!\n")

print("📊 VARIÁVEIS CRIADAS:")
print("   1. total_erbs_4g")
print("   2. erbs_per_1000hab (DOSE PRINCIPAL)")
print("   3. erbs_per_km2")
print("   4. intensidade_4g_norm")
print("   5. categoria_4g")
print("   6. alta_intensidade_4g")
print()

print("🎯 COBERTURA:")
print(f"   {pct_com_dados:.1f}% das observações têm dados de intensidade 4G")
print()

print("📈 PRÓXIMOS PASSOS:")
print("   1. Carregar painel em R")
print("   2. Estimar dose-response com Callaway-Sant'Anna")
print("   3. Análise de heterogeneidade")
print()

print("💡 EXEMPLO DE CÓDIGO R:")
print("""
library(tidyverse)
library(did)

# Carregar dados
dados <- read_csv("painel_com_intensidade_4g.csv",
                  col_types = cols(codigo_tse = col_character()))

# Dose-response
cs_dose <- att_gt(
  yname = "esteban_ray",
  tname = "tempo",
  idname = "id",
  gname = "gname",
  xformla = ~ anos_3g + erbs_per_1000hab,  # DOSE!
  data = dados,
  control_group = "notyettreated"
)

aggte(cs_dose, type = "simple")
""")

RESUMO FINAL

✅ PROCESSAMENTO CONCLUÍDO COM SUCESSO!

📊 VARIÁVEIS CRIADAS:
   1. total_erbs_4g
   2. erbs_per_1000hab (DOSE PRINCIPAL)
   3. erbs_per_km2
   4. intensidade_4g_norm
   5. categoria_4g
   6. alta_intensidade_4g

🎯 COBERTURA:
   100.0% das observações têm dados de intensidade 4G

📈 PRÓXIMOS PASSOS:
   1. Carregar painel em R
   2. Estimar dose-response com Callaway-Sant'Anna
   3. Análise de heterogeneidade

💡 EXEMPLO DE CÓDIGO R:

library(tidyverse)
library(did)

# Carregar dados
dados <- read_csv("painel_com_intensidade_4g.csv",
                  col_types = cols(codigo_tse = col_character()))

# Dose-response
cs_dose <- att_gt(
  yname = "esteban_ray",
  tname = "tempo",
  idname = "id",
  gname = "gname",
  xformla = ~ anos_3g + erbs_per_1000hab,  # DOSE!
  data = dados,
  control_group = "notyettreated"
)

aggte(cs_dose, type = "simple")

